## player前処理
#### 15
- 2017年のデータを1/4ずつ使う

#### 14
- コースの種類のみ

#### 13
- コースの種類

#### 12
- 球団ごとの年棒ランク
- 選手IDごとの2017年のコース実績

In [104]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import feather
pd.set_option('display.max_Columns', 100)

In [105]:
train_player = pd.read_feather('data/train_player.f')
test_player = pd.read_feather('data/test_player.f')
print(train_player.shape)
print(test_player.shape)

(911, 25)
(1846, 25)


In [106]:
all_player = train_player.append(test_player, ignore_index=True)
print(all_player.shape)

(2757, 25)


### 2017年のn/4

In [107]:
No = 4

In [108]:
OUT_PITCHER = 'intermediate/all_pitcher_15_{}.f'.format(No)
OUT_ALLPLAYER = 'intermediate/all_player_15_{}.f'.format(No)

### 2017年の成績(1/4ずつ)

In [109]:
pit_2017 = pd.read_feather('intermediate/pit_2017_7_{}.f'.format(No))
bat_2017 = pd.read_feather('intermediate/bat_2017_7_{}.f'.format(No))
print(pit_2017.shape)
print(bat_2017.shape)

(514, 27)
(342, 3)


### 外国人助っ人

In [110]:
all_player['foreigner']=0
all_player.loc[all_player['出身国']!='日本', 'foreigner'] = 1

### 高卒・大卒・社会人

In [111]:
# 社会人出身
all_player['company'] = 0
all_player.loc[~all_player['社会人'].isnull(), 'company'] = 1
# 大卒
all_player['univ']=0
all_player.loc[all_player['出身大学ID']!=0, 'univ'] = 1
# 高卒
all_player['highsch'] = 0
all_player.loc[(all_player['company']==0)&(all_player['univ']==0)&(all_player['foreigner']==0) , 'highsch'] = 1

### 年齢、現役年数

In [112]:
# 年齢
all_player['birth_day'] = pd.to_datetime(all_player['生年月日'])
all_player['age'] = all_player['年度'] - all_player['birth_day'].dt.year
# 現役年数
all_player['play_year'] = all_player['年度'] - all_player['ドラフト年']
all_player.loc[all_player['ドラフト年'].isnull(), 'play_year'] = 6

### 年棒

#### 球団ごとの年棒順位

In [113]:
all_player['salary_rank'] = all_player.groupby(['年度', 'チームID'])['年俸'].rank(ascending=False)

In [114]:
all_player['rank_year'] = all_player['salary_rank']/ all_player['play_year'] 
all_player['rank_x_year'] = all_player['salary_rank'] * all_player['play_year'] 
all_player['salary_year'] = all_player['年俸']/ all_player['play_year'] 
all_player['salary_x_year'] = all_player['年俸'] * all_player['play_year'] 

### 身長・体重
BMI=体重/身長^2

In [115]:
all_player['bmi'] = all_player['体重']*10000/(all_player['身長']*all_player['身長'])

### 不要な列を削除

In [116]:
all_player.drop(
    columns=[
        'チームID', 'チーム名', '選手名', '背番号', '打', '生年月日', 
        '出身高校ID', '出身高校名', '出身大学ID', '出身大学名', '社会人', 
        'ドラフト年', 'ドラフト種別', 
        '出身国', '出身地', '血液型', 'birth_day',
    ], inplace=True)

### rename

In [117]:
all_player.rename(columns={
    '育成選手F': 'firm',
    '身長': 'height',
    '体重': 'weight',
    'ドラフト順位': 'draft_order',
    '年俸': 'salary',
}, inplace=True)

### 投手のみ

In [118]:
all_pitcher = all_player[all_player['位置']=='投手']

In [119]:
dummy = pd.DataFrame({
    '投': ['右', '右', '左', '左'],
    'pit_bat': ['R_L', 'R_R', 'L_L', 'L_R']
})
all_pitcher = all_pitcher.merge(dummy, on='投', how='outer')

In [120]:
all_pitcher = all_pitcher.merge(pit_2017, left_on=['選手ID','pit_bat'], right_on=['投手ID','pit_bat'], how='left')

In [121]:
all_pitcher.loc[(all_pitcher['投手ID'].isnull()) & (all_pitcher['foreigner']==1), '投手ID'] = -1
all_pitcher.loc[all_pitcher['投手ID'].isnull(), '投手ID'] = 0

#### 情報がない選手
- 外国人投手-> 投手ID=-1
- 日本人投手-> 投手ID=0
- 2017の平均で穴埋め

In [122]:
def fill_ball(condition, source, target):
    ball_kind = [
        'straight', 'curve', 'slider', 'shoot', 'fork', 'changeup', 'sinker', 'cutball', 'total',
        'pit_game_cnt', 'pit_inning_cnt', 'pit_batter_cnt',
        'course_0', 'course_1', 'course_2', 'course_3', 'course_4', 'course_5', 'course_6', 
        'course_7', 'course_8', 'course_9', 'course_10', 'course_11', 'course_12'
    ]
    for ball in ball_kind:
        target.loc[condition, ball] = source[ball]

In [123]:
all_pitcher[(all_pitcher['投手ID']==0)]

,年度,選手ID,firm,位置,投,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,pit_bat,投手ID,straight,curve,slider,shoot,fork,changeup,sinker,cutball,total,course_0,course_1,course_2,course_3,course_4,course_5,course_6,course_7,course_8,course_9,course_10,course_11,course_12,pit_game_cnt,pit_inning_cnt,pit_batter_cnt
0,2017,12107,0,投手,左,175,82,3.0,5000,0,1,0,0,37,16.0,23.5,1.468750,376.0,312.500000,80000.0,26.775510,L_L,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,12107,0,投手,左,175,82,3.0,5000,0,1,0,0,37,16.0,23.5,1.468750,376.0,312.500000,80000.0,26.775510,L_R,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2017,700100,0,投手,左,172,66,4.0,12000,0,1,0,0,31,11.0,16.0,1.454545,176.0,1090.909091,132000.0,22.309356,L_L,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2017,700100,0,投手,左,172,66,4.0,12000,0,1,0,0,31,11.0,16.0,1.454545,176.0,1090.909091,132000.0,22.309356,L_R,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,2017,1000101,0,投手,左,176,82,3.0,950,0,0,1,0,29,7.0,52.0,7.428571,364.0,135.714286,6650.0,26.472107,L_L,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2825,2019,1800119,0,投手,右,188,84,3.0,600,0,0,0,1,19,1.0,58.5,58.500000,58.5,600.000000,600.0,23.766410,R_R,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2826,2019,1800125,1,投手,右,190,84,1.0,250,0,0,0,1,19,1.0,75.5,75.500000,75.5,250.000000,250.0,23.268698,R_L,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2827,2019,1800125,1,投手,右,190,84,1.0,250,0,0,0,1,19,1.0,75.5,75.500000,75.5,250.000000,250.0,23.268698,R_R,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2828,2019,1800126,1,投手,右,174,79,2.0,250,0,1,1,0,25,1.0,75.5,75.500000,75.5,250.000000,250.0,26.093275,R_L,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 日本人平均

In [124]:
RightLeft = ['R_L', 'R_R', 'L_R', 'L_L']
for RL in RightLeft:
    pit_mean = all_pitcher[(all_pitcher['foreigner']==0)&(all_pitcher['投手ID']!=0)&(all_pitcher['pit_bat']==RL)].mean()
    condition = (all_pitcher['投手ID']==0)&(all_pitcher['pit_bat']==RL)
    fill_ball(condition, pit_mean, all_pitcher)

#### 外国人平均

In [125]:
for RL in RightLeft:
    pit_mean = all_pitcher[(all_pitcher['foreigner']==1)&(all_pitcher['投手ID']!=-1)&(all_pitcher['pit_bat']==RL)].mean()
    condition = (all_pitcher['投手ID']==-1)&(all_pitcher['pit_bat']==RL)
    fill_ball(condition, pit_mean, all_pitcher)

### 各球種のストレートに対する比率

In [126]:
ball_not_straight = ['curve', 'slider', 'shoot', 'fork', 'changeup', 'sinker', 'cutball']
for ball in ball_not_straight:
    all_pitcher[ball] = all_pitcher[ball] / all_pitcher['straight']

### コースの比率

In [127]:
course_kind = ['course_0', 'course_1', 'course_2', 'course_3', 'course_4', 'course_5', 'course_6', 
                        'course_7', 'course_8', 'course_9', 'course_10', 'course_11', 'course_12']
for course in course_kind:
    all_pitcher[course] = all_pitcher[course] / all_pitcher['total']

### コースの種類

In [128]:
all_pitcher['high_str'] = all_pitcher['course_0'] + all_pitcher['course_3'] + all_pitcher['course_6'] 
all_pitcher['high_ball'] = all_pitcher['course_9'] + all_pitcher['course_10'] 
all_pitcher['mid_str'] = all_pitcher['course_1'] + all_pitcher['course_4'] + all_pitcher['course_7'] 
all_pitcher['low_str'] = all_pitcher['course_2'] + all_pitcher['course_5'] + all_pitcher['course_8'] 
all_pitcher['low_ball'] = all_pitcher['course_11'] + all_pitcher['course_12'] 

In [129]:
all_pitcher['left_str'] = all_pitcher['course_0'] + all_pitcher['course_1'] + all_pitcher['course_2'] 
all_pitcher['left_ball'] = all_pitcher['course_9'] + all_pitcher['course_11'] 
all_pitcher['center_str'] = all_pitcher['course_3'] + all_pitcher['course_4'] + all_pitcher['course_5'] 
all_pitcher['right_str'] = all_pitcher['course_6'] + all_pitcher['course_7'] + all_pitcher['course_8'] 
all_pitcher['right_ball'] = all_pitcher['course_10'] + all_pitcher['course_12'] 

### 不要な列を削除

In [130]:
all_pitcher.drop(columns=[
    'straight', '投手ID', '位置', '投',
    'course_0', 'course_1', 'course_2', 'course_3', 'course_4', 'course_5', 'course_6', 
    'course_7', 'course_8', 'course_9', 'course_10', 'course_11', 'course_12'
], inplace=True)

In [131]:
all_pitcher.head()

,年度,選手ID,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,pit_bat,curve,slider,shoot,fork,changeup,sinker,cutball,total,pit_game_cnt,pit_inning_cnt,pit_batter_cnt,high_str,high_ball,mid_str,low_str,low_ball,left_str,left_ball,center_str,right_str,right_ball
0,2017,12107,0,175,82,3.0,5000,0,1,0,0,37,16.0,23.5,1.468750,376.0,312.500000,80000.0,26.775510,L_L,0.101676,0.555957,0.063635,0.085918,0.086319,0.005420,0.062632,104.508021,6.839572,17.31016,26.513369,0.136264,0.175357,0.188507,0.173054,0.326818,0.233127,0.355166,0.161183,0.103515,0.147009
1,2017,12107,0,175,82,3.0,5000,0,1,0,0,37,16.0,23.5,1.468750,376.0,312.500000,80000.0,26.775510,L_R,0.117758,0.355693,0.128445,0.145651,0.269257,0.029641,0.042546,166.203209,6.839572,17.31016,41.368984,0.126384,0.193147,0.179762,0.159427,0.341281,0.152928,0.253153,0.154698,0.157947,0.281274
2,2017,400010,0,186,95,1.0,20000,0,1,0,0,35,14.0,10.0,0.714286,140.0,1428.571429,280000.0,27.459822,L_L,0.000000,0.857143,1.071429,0.071429,0.714286,0.000000,0.214286,55.000000,2.000000,7.00000,14.000000,0.072727,0.090909,0.254545,0.200000,0.381818,0.145455,0.236364,0.200000,0.181818,0.236364
3,2017,400010,0,186,95,1.0,20000,0,1,0,0,35,14.0,10.0,0.714286,140.0,1428.571429,280000.0,27.459822,L_R,0.000000,1.090909,2.000000,0.272727,1.818182,0.000000,0.000000,68.000000,2.000000,7.00000,18.000000,0.088235,0.161765,0.132353,0.176471,0.441176,0.044118,0.220588,0.102941,0.250000,0.382353
4,2017,600098,0,184,88,1.0,32000,0,0,0,1,34,12.0,2.0,0.166667,24.0,2666.666667,384000.0,25.992439,L_L,0.000000,2.750000,1.750000,0.000000,0.000000,0.000000,0.000000,22.000000,5.000000,5.00000,7.000000,0.090909,0.000000,0.181818,0.272727,0.454545,0.409091,0.363636,0.090909,0.045455,0.090909


### 投手のみ出力

In [132]:
all_pitcher.to_feather(OUT_PITCHER)

### 打者(全選手)

In [133]:
all_player = all_player.merge(bat_2017, left_on='選手ID', right_on='打者ID', how='left')

In [134]:
all_player.loc[all_player['打者ID'].isnull(), '打者ID'] = 0
# 投手以外
bat_mean = all_player[(all_player['打者ID']!=0)&(all_player['位置']!='投手')].mean()
condition = (all_player['打者ID']==0)&(all_player['位置']!='投手')
all_player.loc[condition, 'batter_cnt'] = bat_mean['batter_cnt']
all_player.loc[condition, 'bat_game_cnt'] = bat_mean['bat_game_cnt']
# 投手
bat_mean = all_player[(all_player['打者ID']!=0)&(all_player['位置']=='投手')].mean()
condition = (all_player['打者ID']==0)&(all_player['位置']=='投手')
all_player.loc[condition, 'batter_cnt'] = bat_mean['batter_cnt']
all_player.loc[condition, 'bat_game_cnt'] = bat_mean['bat_game_cnt']

### 不要な列を削除

In [135]:
all_player.drop(
    columns=['打者ID', '位置', '投'], inplace=True)

In [136]:
all_player.head()

,年度,選手ID,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,batter_cnt,bat_game_cnt
0,2017,11343,0,183,86,5.0,4500,0,0,0,1,41,23.0,26.5,1.152174,609.5,195.652174,103500.0,25.680074,6.000000,6.000000
1,2017,11726,0,177,85,1.0,3700,0,0,0,1,36,19.0,28.5,1.500000,541.5,194.736842,70300.0,27.131412,70.266404,20.476378
2,2017,12049,0,180,97,1.0,26000,0,0,1,0,38,17.0,4.0,0.235294,68.0,1529.411765,442000.0,29.938272,163.000000,39.000000
3,2017,12107,0,175,82,3.0,5000,0,1,0,0,37,16.0,23.5,1.468750,376.0,312.500000,80000.0,26.775510,8.676136,4.022727
4,2017,12179,0,184,94,1.0,7000,0,0,0,1,34,16.0,20.0,1.250000,320.0,437.500000,112000.0,27.764650,8.676136,4.022727


### 全選手出力

In [137]:
all_player.to_feather(OUT_ALLPLAYER)